In [35]:
# libraries
import pandas as pd
import random

# local modules
import preprocessing as pp

# reload changes in modules automatically
%reload_ext autoreload
%autoreload 2

## Load and preprocess data

In [2]:
messages = pp.preprocess()
messages

,id,operator,timestamp,category,text,language,translated-text
0,2000000060,H2851RM1,2015-11-25 18:35:21+00:00,ZSD_SR_AA,"Hallo zusammen aktuell sind die Systeme i06, i...",de,"Hallo zusammen aktuell sind die Systeme i06, i..."
1,2000000081,H2851RM1,2015-12-01 09:06:58+00:00,ZSD_SR_IDES,"Wie bereits früher in Mails dokumentiert, läuf...",de,"Wie bereits früher in Mails dokumentiert, läuf..."
2,2000000091,H2851RM1,2015-12-03 16:13:00+00:00,ZSD_SR_IDES,Kontextmenü im Workflow Builder funktioniert n...,de,Kontextmenü im Workflow Builder funktioniert n...
3,2000000092,H2851RM1,2015-12-03 16:38:37+00:00,ZSD_SR_IDES,Mehrstufige Container-Ausdrücke in der Aufgabe...,de,Mehrstufige Container-Ausdrücke in der Aufgabe...
4,2000000093,H2851RM1,2015-12-03 16:58:37+00:00,ZSD_SR_ES,Aufruf des neueren Editors zum Bearbeiten von ...,de,Aufruf des neueren Editors zum Bearbeiten von ...
...,...,...,...,...,...,...,...
6054,2000012394,H2851SF2,2021-06-01 15:42:36+00:00,ZSD_SR_GBI,"Hi PERSON2112>, das ECC 6.08 GBI 3.20 Exklusiv...",de,"Hi PERSON2112>, das ECC 6.08 GBI 3.20 Exklusiv..."
6055,2000012395,H2851US4,2021-06-02 09:04:00+00:00,ZSD_SR_ES,"Good morning, please generate dev keys. for Us...",en,"Guten Morgen, bitte generieren Sie Entwicklers..."
6056,2000012396,H2851SF2,2021-06-02 14:44:10+00:00,ZSD_SR_BW,"Sehr geehrtes SAP UCC Support Team, in einer Ü...",de,"Sehr geehrtes SAP UCC Support Team, in einer Ü..."
6057,2000012398,H2851BL1,2021-06-03 10:25:44+00:00,ZSD_SR_KTV,"My colleague, PERSON2113 (<PERSON2112>), canno...",en,"Mein Kollege, PERSON2113 (<PERSON2112> ), wede..."


## Train and evaluate model

In [155]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, top_k_accuracy_score

x = messages['text']
y = messages['operator']  # change the prediction target here e.g. 'operator' or 'category'

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

cv = CountVectorizer(stop_words=pp.NLP.Defaults.stop_words)
x_train_tf = cv.fit_transform(x_train)

tfidf_transformer = TfidfTransformer()
x_train_tfidf= tfidf_transformer.fit_transform(x_train_tf)

mnb = MultinomialNB()
mnb = mnb.fit(x_train_tfidf, y_train)

pred = mnb.predict(x_train_tfidf)
print(f'TRAIN Accuracy: {accuracy_score(y_train, pred)*100:.1f}%')

x_test_tf = cv.transform(x_test)
x_test_tfidf = tfidf_transformer.transform(x_test_tf)

pred = mnb.predict(x_test_tfidf)
print(f'TEST Accuracy: {accuracy_score(y_test, pred)*100:.1f}%')

pred = mnb.predict_proba(x_test_tfidf)
print(f'TEST Top 3 Accuracy: {top_k_accuracy_score(y_test, pred, k=3)*100:.1f}%')

TRAIN Accuracy: 32.3%
TEST Accuracy: 26.4%
TEST Top 3 Accuracy: 44.7%


## Demo with one message

In [162]:
i = random.randint(0, len(messages)-1)
x = messages['translated-text'][i:i+1]
y = messages['operator'][i:i+1]
print(x[i])

Alert-DetailsName....................Java-Instanz nicht verfügbar Startzeit...............30.01.2017 17:41:50 CET Endezeit................30.01.2017 17:41:50 CET Verwaltetes Objekt......I8300001~JAVA~i83lp1_I83_00 Typ des verwalteten ObjektsTechnische Instanz Bewertung...............Rot Kategorie...............Verfügbarkeit Status..................Offen Alert-Beschreibung Die J2EE-Instanz i83lp1_I83_00 oder Teile von ihr laufen nicht. Prüfen S = ie die Alert-Details, um festzustellen, welche Komponenten nicht antwort = en. Starten Sie die Instanz gegebenenfalls neu. Analysewerkzeuge: Instanzstatus in http://i04lp1.informatik.tu-muenchen.de:50413/sapmc/sap = mc.html?SID=I83<(>&<)>NR=00<(>&<)>HOST=i83lp1 anzeigen. Überprüfen Sie manuell die http://i83lp1.informatik.tu-muenchen.de:50000 = index.html">HTTP-Verfügbarkeit der Instanz<(><<)>/a>.


In [163]:
x_tf = cv.transform(x)
x_tfidf = tfidf_transformer.transform(x_tf)
print(f'Predicted: {mnb.predict(x_tfidf)[0]}')
print(f'True: {y[i]}')

Predicted: H2851AS9
True: H2851AS9
